In [1]:
#Import Required Python, Spark Libraries
from itertools import islice
import six
from pyspark import SparkContext, SparkConf
#Define Spark configuration and SparkContext
conf = SparkConf().setAppName("Simple Masking App").setMaster("local")
sc = SparkContext(conf=conf)

# Option 1: When Masking Should be done Only on email

In [2]:
#Define Python Function to Mask email 

def maskEmail(email):
    firstPart = list(email.split('@')[0])
    secondPart = list(email.split('@')[1])
    f = ['*' for sub in firstPart[1:]] 
    s = ['*' for sub in secondPart[1:"".join(secondPart).find('.')]] 
    str1="".join(firstPart[0:1])+"".join(f)
    str2="".join(secondPart[0:1])+"".join(s)+"".join(secondPart["".join(secondPart).find('.'):])
    return str1+'@'+str2

#Test Above function
str = 'james@gmail.com'
maskEmail(str)

'j****@g****.com'

In [3]:
#Read file data in Spark RDD
empRDD = sc.textFile("EmployData.csv")
#Remove Header
empNoHeaderRDD = empRDD.mapPartitionsWithIndex(lambda idx, it: islice(it, 1, None) if idx == 0 else it)
#Print RDD data
for i in empNoHeaderRDD.take(10):print(i)

James,james@gmail.com,Sr. Developer
Smith,Smith@gmail.com,Project Lead


In [4]:
#Apply Function maskEmail() on email stored in RDD.
empRDD1 = empNoHeaderRDD.map(lambda x:(x.split(",")[0],maskEmail(x.split(",")[1]),x.split(",")[2]))
for i in empRDD1.take(2):print(i)

('James', 'j****@g****.com', 'Sr. Developer')
('Smith', 'S****@g****.com', 'Project Lead')


# Option 2: When Masking Should be done On Multiple Columns

In [5]:
#Define Python Function to Mask column 
def maskColumn(colName):
    if isinstance(colName, six.string_types) == True:
        return '*****Masked Data*****'
    else:
        return 0

In [6]:
maskColumn("Confidential Data")

'*****Masked Data*****'

In [7]:
maskColumn(35440)

0

In [8]:
maskColumn(35440.766)

0

In [9]:
empRDD1 = empNoHeaderRDD.map(lambda x:(x.split(",")[0],maskColumn(x.split(",")[1]),maskColumn(x.split(",")[2])))
for i in empRDD1.take(2):print(i)

('James', '*****Masked Data*****', '*****Masked Data*****')
('Smith', '*****Masked Data*****', '*****Masked Data*****')


# Option 3: First Masking And Later Unmasking Should be done

In [30]:
#In this case, Str to Hex conversion can be used to mask data. To Unmask such data, Hex to Str conversion can be used.

import functools
def toHex(s):
    lst = []
    for ch in s:
        hv = hex(ord(ch)).replace('0x', '')
        if len(hv) == 1:
            hv = '0'+hv
        lst.append(hv)
        
    return functools.reduce(lambda x,y:x+y, lst)

#convert hex repr to string
def toStr(s):
    return s and chr(int(s[:2], base=16)) + toStr(s[2:]) or ''

In [33]:
toHex('Test Data')

'546573742044617461'

In [35]:
toStr('546573742044617461')

'Test Data'